In [1]:
    # MTG 1/6
    #POStagとENTtagの間には何らかの関係性が存在？
    #NERにおけるPOStagアプローチはsimpleと何も変わらないのでは＝＞誤差レベルの違いである可能性が高い
    #構文木のほうにいけるとよいのですが...

In [28]:
import torch
device = torch.device("cuda:1")

# data 拡張

## train

In [8]:
with open('data_original/train.txt', 'r', encoding='UTF-8') as f:
    data = f.read()
    data = data.split("\n\n")
    data = data[1:14987] #不要部分をカット

In [9]:
# storing conll format data in multiple list
document = []
for sentence in data:#14988 sentences
    
    new_sentence_list = []
    sentence_list = sentence.split("\n")
    
    for token in sentence_list:
        token_list = token.split(" ")
        new_sentence_list.append(token_list)
    document.append(new_sentence_list)
    
    # [[['-DOCSTART-', '-X-', '-X-', 'O']],
    #  [['EU', 'NNP', 'B-NP', 'B-ORG'],
    #   ['rejects', 'VBZ', 'B-VP', 'O'],
    #   ['German', 'JJ', 'B-NP', 'B-MISC'],...

In [10]:
# ['B-ADVP', 'I-ADVP', 'I-ADJP', 'I-SBAR',
#  'I-PP', 'B-LST', 'B-SBAR', 'B-NP',
#  'B-ADJP', 'I-INTJ', 'B-VP', '-X-',
#  'I-CONJP', 'B-PRT', 'B-CONJP', 'I-LST',
#  'B-INTJ', 'B-PP', 'I-VP', 'O', 'I-NP']

# I-B I-other cut
#    I-I stay 
# B-I
#   B-B B-oter cut

In [11]:
new_document = []

for sentence in document:
    sentence_list = []
    
    for i in range(len(sentence)-1):
        sentence_list.append(sentence[i])
        
        if "I-" in sentence[i][2]:
            if "I-" in sentence[i+1][2]:
                pass
            else: #B,other
                sentence_list.append(-1)
                
        else: #B,other
            if "I-" in sentence[i+1][2]:
                pass
            else: #B,other
                sentence_list.append(-1)
    
    final_sentence_list = []
    new_tokens_list = []
    all_token = []
    syntactic_tag = ""
    bio_tag = []
    
    for token in sentence_list:
        if token == -1 or token==sentence_list[-1]:
            
            new_tokens_list.append(tuple(all_token))
            new_tokens_list.append(tuple([syntactic_tag,tuple(bio_tag)]))#タグ生成
            
            final_sentence_list.append(new_tokens_list)
            
            new_tokens_list = []
            all_token = []
            syntactic_tag = ""
            bio_tag = []
        else:
            all_token.append(token[0])
            if syntactic_tag == "":
                syntactic_tag=token[2]#タグ生成
            bio_tag.append(token[3])
    new_document.append(final_sentence_list)

In [12]:
new_document[0:2]

[[[('EU',), ('B-NP', ('B-ORG',))],
  [('rejects',), ('B-VP', ('O',))],
  [('German', 'call'), ('B-NP', ('B-MISC', 'O'))],
  [('to', 'boycott'), ('B-VP', ('O', 'O'))],
  [('British', 'lamb'), ('B-NP', ('B-MISC', 'O'))]],
 [[(), ('', ())]]]

In [13]:
# とりあえず，一ブロックに一単語のみ格納されているケースは削除
# 2以上のケース　かつ　ブロック内のBOIタグ構造が完全に一致するものだけを交換

In [14]:
# making dictonary object with label as key and token as value
label_token_dic = {}
cnt = 0
for sentence in new_document:
    for block in sentence:
        if len(block[0])!=1:# block内にトークン2以上の時のみ辞書登録
            tokens = block[0]
            label = block[1]
            cnt = cnt + 1
            if label not in label_token_dic:
                label_token_dic[label] = set()
            label_token_dic[label].add(tuple(tokens))
        
    # {('B-NP', ('B-MISC', 'O')): {('A$', '1'),
    #   ('A$', '100'),
    #   ('A$', '14,000'),
    #   ('A$', '28,000'),...
    
    # label_token_dic.keys() でkey一覧取得可　個数：663
    # cnt　に合計要素数格納　個数：41377

In [15]:
# making BI label list
# BIを含むもののみ交換

BI_label_list = []
for label in list(label_token_dic.keys()):
    NG_set = [{"O"},set(),{"-X-"},{"O","-X-"}]
    if set(label[1]) not in NG_set:
        BI_label_list.append(label)
        
# [('B-NP', ('B-MISC', 'O')),
#  ('B-NP', ('O', 'B-ORG', 'I-ORG')),
#  ('B-NP', ('O', 'O', 'O', 'B-PER', 'I-PER')),...

In [29]:
def Sent2str(sentence_list):
    return " ".join([token_list[0] for token_list in sentence_list])

def Sent2str_for_syntactic(sentence_list):
    sentence_str = ""
    for block in sentence_list:
        for token in block[0]:
            sentence_str = sentence_str+" "+token
    return sentence_str

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

In [34]:
import random
import csv


def SyntacticDA_BI(n,filename):
    data_f_path = "data_augment/SyntacticDA_BI/"+str(n)+"_"+filename+".txt"
    memo_f_path = "data_augment/SyntacticDA_BI/"+str(n)+"_"+filename+"_memo.txt"
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for now in range(n):
            if now%10 == 0:
                print('\r%d / %d' %(now, n), end='')

            x = random.randint(0,len(new_document)-1)
            origin_sentence = new_document[x]
            aug_sentence = []
            cnt = 0

            for block_list in origin_sentence:
                tokens = block_list[0]
                label = block_list[1]
                origin_labels = block_list[1][1]

                if label in BI_label_list:
                    if len(label_token_dic[label])!=1:
                        tokens=random_choice_token(label_token_dic[label]-{tokens})
                        cnt = cnt + 1
                    
                for i in range(len(tokens)):
                    aug_sentence.append([tokens[i],origin_labels[i]])
                    data_f.write(" ".join([tokens[i],origin_labels[i]])+"\n")
            data_f.write("\n")
            \
            memo_f.write(str(x)+"\t"+Sent2str_for_syntactic(new_document[x])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+Sent2str(aug_sentence)+"\n")
            memo_f.write("\n")
        
        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [35]:
# #train
n = 15000

In [37]:
SyntacticDA_BI(n*1,"train")
SyntacticDA_BI(n*4,"train")
SyntacticDA_BI(n*9,"train")

134990 / 135000

## valid 

In [39]:
with open('data_original/valid.txt', 'r', encoding='UTF-8') as f:
    data = f.read()
    data = data.split("\n\n")
    data = data[1:3466]

In [40]:
# 以下同様
document = []
for sentence in data:#14988 sentences
    
    new_sentence_list = []
    sentence_list = sentence.split("\n")
    
    for token in sentence_list:
        token_list = token.split(" ")
        new_sentence_list.append(token_list)
    document.append(new_sentence_list)



new_document = []
for sentence in document:
    sentence_list = []
    
    for i in range(len(sentence)-1):
        sentence_list.append(sentence[i])
        
        if "I-" in sentence[i][2]:
            if "I-" in sentence[i+1][2]:
                pass
            else: #B,other
                sentence_list.append(-1)
                
        else: #B,other
            if "I-" in sentence[i+1][2]:
                pass
            else: #B,other
                sentence_list.append(-1)
    
    final_sentence_list = []
    new_tokens_list = []
    all_token = []
    syntactic_tag = ""
    bio_tag = []
    
    for token in sentence_list:
        if token == -1 or token==sentence_list[-1]:
            
            new_tokens_list.append(tuple(all_token))
            new_tokens_list.append(tuple([syntactic_tag,tuple(bio_tag)]))#タグ生成
            
            final_sentence_list.append(new_tokens_list)
            
            new_tokens_list = []
            all_token = []
            syntactic_tag = ""
            bio_tag = []
        else:
            all_token.append(token[0])
            if syntactic_tag == "":
                syntactic_tag=token[2]#タグ生成
            bio_tag.append(token[3])
    new_document.append(final_sentence_list)

    

label_token_dic = {}
cnt = 0
for sentence in new_document:
    for block in sentence:
        if len(block[0])!=1:# block内にトークン2以上の時のみ辞書登録
            tokens = block[0]
            label = block[1]
            cnt = cnt + 1
            if label not in label_token_dic:
                label_token_dic[label] = set()
            label_token_dic[label].add(tuple(tokens))


BI_label_list = []
for label in list(label_token_dic.keys()):
    NG_set = [{"O"},set(),{"-X-"},{"O","-X-"}]
    if set(label[1]) not in NG_set:
        BI_label_list.append(label)

In [41]:
#valid
n = 3500

In [42]:
SyntacticDA_BI(n*1,"valid")
SyntacticDA_BI(n*4,"valid")
SyntacticDA_BI(n*9,"valid")

31490 / 31500